In [4]:
import pandas as pd
from pathlib import Path

data_dir = Path("./CSV-data")
csv_files = sorted([f for f in data_dir.glob("*.csv")])
dfs = []
for csv_file in csv_files:
    print(f"processing file: {csv_file}")
    try:
        df = pd.read_csv(csv_file)
        dfs.append(df)
    except Exception as e:
        print(f"Error parsing {csv_file}: {str(e)}")

processing file: CSV-data/chunk-data.csv
processing file: CSV-data/relation__cityGroup.csv


In [5]:
import openai
import os
api_key_path = "./apiKey.txt"

with open(api_key_path, "r") as f:
  api_key = ' '.join(f.readlines())
  openai.api_key = api_key

os.environ['OPENAI_API_KEY'] = openai.api_key

In [6]:
tableinfo_dir = "LEI_TableInfo"
!mkdir {tableinfo_dir}

mkdir: LEI_TableInfo: File exists


#### Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_RelationshipType, periods, qualifiers, quantifiers, and registration details. Relationship_EndNode_NodeID is the foreign key in the table Relationship_Information. LEI column is the primary key in the table Entity_Legal_Info which is the foreign key Relationship_EndNode_NodeID column in the table Relationship_Information. Relationship_RelationshipType column contains the information about the relation between the given company name. There are 4 type of relations in Relationship_RelationshipType column 'IS_DIRECTLY_CONSOLIDATED_BY', 'IS_ULTIMATELY_CONSOLIDATED_BY', 'IS_FUND-MANAGED_BY' and 'IS_FUND-MANAGER'. Please find the description of all the relations below: 'IS_ULTIMATELY_CONSOLIDATED_BY' - is the ultimate child of given company. 'IS_DIRECTLY_CONSOLIDATED_BY' - is the direct child of given company. 'IS_FUND-MANAGED_BY' - is a relation who is managing the fund of given company. 'IS_FUND-MANAGER' - is a relation who's funds are managed by given company."

In [7]:
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.llms.openai import OpenAI


class TableInfo(BaseModel):
    """Information regarding a structured table."""

    table_name: str = Field(
        ..., description="table name (must be underscores and NO spaces)"
    )
    table_summary: str = Field(
        ..., description="short, concise summary/caption of the table"
    )


prompt_str = """\
Give me a summary of the table with the following JSON format.

- The table name must be unique to the table and describe it while being concise. 
- Do NOT output a generic table name (e.g. table, my_table).

Do NOT make the table name one of the following: {exclude_table_name_list}

Table:
{table_str}

Summary: """

program = LLMTextCompletionProgram.from_defaults(
    output_cls=TableInfo,
    llm=OpenAI(model="gpt-3.5-turbo"),
    prompt_template_str=prompt_str,
)

In [98]:
import json


def _get_tableinfo_with_index(idx: int) -> str:
    results_gen = Path(tableinfo_dir).glob(f"{idx}_*")
    results_list = list(results_gen)
    print(results_list)
    if len(results_list) == 0:
        return None
    elif len(results_list) == 1:
        path = results_list[0]
        return TableInfo.parse_file(path)
    else:
        raise ValueError(
            f"More than one file matching index: {list(results_gen)}"
        )

In [99]:
table_names = set()
table_infos = []
for idx, df in enumerate(dfs):
    table_info = _get_tableinfo_with_index(idx)
    print(table_info)
    if table_info:
        table_infos.append(table_info)
    else:
        while True:
            df_str = df.head(10).to_csv()
            table_info = program(
                table_str=df_str,
                exclude_table_name_list=str(list(table_names)),
            )
            table_name = table_info.table_name
            print(f"Processed table: {table_name}")
            if table_name not in table_names:
                table_names.add(table_name)
                break
            else:
                # try again
                print(f"Table name {table_name} already exists, trying again.")
                pass

        out_file = f"{tableinfo_dir}/{idx}_{table_name}.json"
        json.dump(table_info.dict(), open(out_file, "w"))
        table_infos.append(table_info)

[PosixPath('LEI_TableInfo/0_Entity_Legal_Info.json')]
table_name='Entity_Legal_Info' table_summary='Summary of legal information for various entities including legal names, addresses, registration details, entity status, creation dates, and conformity flags.'
[PosixPath('LEI_TableInfo/1_Relationship_Information.json')]
table_name='Relationship_Information' table_summary="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_RelationshipType, periods, qualifiers, quantifiers, and registration details. Relationship_EndNode_NodeID is the foreign key in the table Relationship_Information. LEI column is the primary key in the table Entity_Legal_Info which is the foreign key Relationship_EndNode_NodeID column in the table Relationship_Information. Relationship_RelationshipType column contains the information about the relation between the given company name. There are 4 type of relations in Relationship_RelationshipType column

In [100]:
table_infos

[TableInfo(table_name='Entity_Legal_Info', table_summary='Summary of legal information for various entities including legal names, addresses, registration details, entity status, creation dates, and conformity flags.'),
 TableInfo(table_name='Relationship_Information', table_summary="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_RelationshipType, periods, qualifiers, quantifiers, and registration details. Relationship_EndNode_NodeID is the foreign key in the table Relationship_Information. LEI column is the primary key in the table Entity_Legal_Info which is the foreign key Relationship_EndNode_NodeID column in the table Relationship_Information. Relationship_RelationshipType column contains the information about the relation between the given company name. There are 4 type of relations in Relationship_RelationshipType column 'IS_DIRECTLY_CONSOLIDATED_BY', 'IS_ULTIMATELY_CONSOLIDATED_BY', 'IS_FUND-MANAGED_BY' and

In [11]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
)
import re

In [12]:
# Function to create a sanitized column name
def sanitize_column_name(col_name):
    # Remove special characters and replace spaces with underscores
    return re.sub(r"\W+", "_", col_name)

In [101]:
# Function to create a table from a DataFrame using SQLAlchemy
def create_table_from_dataframe(
    df: pd.DataFrame, table_name: str, engine, metadata_obj
):
    # Sanitize column names
    sanitized_columns = {col: sanitize_column_name(col) for col in df.columns}
    df = df.rename(columns=sanitized_columns)

    # Dynamically create columns based on DataFrame columns and data types
    columns = [
        Column(col, String if dtype == "object" else Integer)
        for col, dtype in zip(df.columns, df.dtypes)
    ]

    # Create a table with the defined columns
    table = Table(table_name, metadata_obj, *columns)

    # Create the table in the database
    metadata_obj.create_all(engine)

    # Insert data from DataFrame into the table
    with engine.connect() as conn:
        for _, row in df.iterrows():
            insert_stmt = table.insert().values(**row.to_dict())
            conn.execute(insert_stmt)
        conn.commit()

In [14]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()
for idx, df in enumerate(dfs):
    tableinfo = _get_tableinfo_with_index(idx)
    print(f"Creating table: {tableinfo.table_name}")
    create_table_from_dataframe(df, tableinfo.table_name, engine, metadata_obj)

[PosixPath('LEI_TableInfo/0_Entity_Legal_Info.json')]
Creating table: Entity_Legal_Info
[PosixPath('LEI_TableInfo/1_Relationship_Information.json')]
Creating table: Relationship_Information


In [102]:
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import SQLDatabase, VectorStoreIndex

sql_database = SQLDatabase(engine)
table_schema_objs = []
table_node_mapping = SQLTableNodeMapping(sql_database)
    
# table_schema_objs = [
#     SQLTableSchema(table_name=t[tindex].table_name, context_str=_get_tableinfo_with_index(tindex).table_summary)
#     for tindex, t in table_infos
# ]  # add a SQLTableSchema for each table

for idx in range(len(table_infos)):
   table_data = _get_tableinfo_with_index(idx)
   table_schema_objs.append(SQLTableSchema(table_name=table_data.table_name, context_str=table_data.table_summary))


obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
obj_retriever = obj_index.as_retriever(similarity_top_k=2)

[PosixPath('LEI_TableInfo/0_Entity_Legal_Info.json')]
[PosixPath('LEI_TableInfo/1_Relationship_Information.json')]


In [103]:
table_schema_objs

[SQLTableSchema(table_name='Entity_Legal_Info', context_str='Summary of legal information for various entities including legal names, addresses, registration details, entity status, creation dates, and conformity flags.'),
 SQLTableSchema(table_name='Relationship_Information', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_RelationshipType, periods, qualifiers, quantifiers, and registration details. Relationship_EndNode_NodeID is the foreign key in the table Relationship_Information. LEI column is the primary key in the table Entity_Legal_Info which is the foreign key Relationship_EndNode_NodeID column in the table Relationship_Information. Relationship_RelationshipType column contains the information about the relation between the given company name. There are 4 type of relations in Relationship_RelationshipType column 'IS_DIRECTLY_CONSOLIDATED_BY', 'IS_ULTIMATELY_CONSOLIDATED_BY', 'IS_FUND-MANAGED_B

In [104]:
from llama_index.core.retrievers import SQLRetriever
from typing import List
from llama_index.core.query_pipeline import FnComponent

sql_retriever = SQLRetriever(sql_database)


def get_table_context_str(table_schema_objs: List[SQLTableSchema]):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        context_strs.append(table_info)
    return "\n\n".join(context_strs)


table_parser_component = FnComponent(fn=get_table_context_str)

In [105]:
from llama_index.core.prompts.default_prompts import DEFAULT_TEXT_TO_SQL_PROMPT
from llama_index.core import PromptTemplate
from llama_index.core.llms import ChatResponse


def parse_response_to_sql(response: ChatResponse) -> str:
    """Parse response to SQL."""
    response = response.message.content
    sql_query_start = response.find("SQLQuery:")
    if sql_query_start != -1:
        response = response[sql_query_start:]
        if response.startswith("SQLQuery:"):
            response = response[len("SQLQuery:") :]
    sql_result_start = response.find("SQLResult:")
    if sql_result_start != -1:
        response = response[:sql_result_start]
    return response.strip().strip("```").strip()


sql_parser_component = FnComponent(fn=parse_response_to_sql)

text2sql_prompt = DEFAULT_TEXT_TO_SQL_PROMPT.partial_format(
    dialect=engine.dialect.name     # sqlite
)
print(text2sql_prompt.template)

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use tables listed below.
{schema}

Question: {query_str}
SQLQuery: 


In [106]:
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n"
    "SQL: {sql_query}\n"
    "SQL Response: {context_str}\n"
    "Response: "
)
response_synthesis_prompt = PromptTemplate(
    response_synthesis_prompt_str,
)

In [81]:
llm=OpenAI(model="gpt-3.5-turbo")

In [50]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
)

qp = QP(
    modules={
        "input": InputComponent(),
        "table_retriever": obj_retriever,
        "table_output_parser": table_parser_component,
        "text2sql_prompt": text2sql_prompt,
        "text2sql_llm": llm,
        "sql_output_parser": sql_parser_component,
        "sql_retriever": sql_retriever,
        "response_synthesis_prompt": response_synthesis_prompt,
        "response_synthesis_llm": llm,
    },
    verbose=True,
)

In [51]:
qp.add_chain(["input", "table_retriever", "table_output_parser"])
qp.add_link("input", "text2sql_prompt", dest_key="query_str")
qp.add_link("table_output_parser", "text2sql_prompt", dest_key="schema")
qp.add_chain(
    ["text2sql_prompt", "text2sql_llm", "sql_output_parser", "sql_retriever"]
)
qp.add_link(
    "sql_output_parser", "response_synthesis_prompt", dest_key="sql_query"
)
qp.add_link(
    "sql_retriever", "response_synthesis_prompt", dest_key="context_str"
)
qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
qp.add_link("response_synthesis_prompt", "response_synthesis_llm")

In [107]:
from llama_index.core import VectorStoreIndex, load_index_from_storage
from sqlalchemy import text
from llama_index.core.schema import TextNode
from llama_index.core import StorageContext
import os
from pathlib import Path
from typing import Dict


def index_all_tables(
    sql_database: SQLDatabase, table_index_dir: str = "table_index_dir"
) -> Dict[str, VectorStoreIndex]:
    """Index all tables."""
    if not Path(table_index_dir).exists():
        os.makedirs(table_index_dir)

    vector_index_dict = {}
    engine = sql_database.engine
    for table_name in sql_database.get_usable_table_names():
        print(f"Indexing rows in table: {table_name}")
        if not os.path.exists(f"{table_index_dir}/{table_name}"):
            # get all rows from table
            with engine.connect() as conn:
                cursor = conn.execute(text(f'SELECT * FROM "{table_name}"'))
                result = cursor.fetchall()
                row_tups = []
                for row in result:
                    row_tups.append(tuple(row))

            # index each row, put into vector store index
            nodes = [TextNode(text=str(t)) for t in row_tups]

            # put into vector store index (use OpenAIEmbeddings by default)
            index = VectorStoreIndex(nodes)

            # save index
            index.set_index_id("vector_index")
            index.storage_context.persist(f"{table_index_dir}/{table_name}")
        else:
            # rebuild storage context
            storage_context = StorageContext.from_defaults(
                persist_dir=f"{table_index_dir}/{table_name}"
            )
            # load index
            index = load_index_from_storage(
                storage_context, index_id="vector_index"
            )
        vector_index_dict[table_name] = index

    return vector_index_dict


vector_index_dict = index_all_tables(sql_database)

Indexing rows in table: Entity_Legal_Info
Indexing rows in table: Relationship_Information


In [108]:
test_retriever = vector_index_dict["Entity_Legal_Info"].as_retriever(
    similarity_top_k=1
)

In [109]:
nodes = test_retriever.retrieve("City group inc")
print(nodes[0].get_content())

(1518211, '6SHGI4ZSSLCXXQSBB395', 'CITIGROUP INC.', 'en', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'en', 'CORPORATION TRUST CENTER 1209 ORANGE ST', None, 'c/o THE CORPORATION TRUST COMPANY', None, None, None, 'WILMINGTON', 'US-DE', 'US', '19801', 'en', '388 GREENWICH STREET', None, None, None, None, 'NEW YORK', 'US-NY', 'US', '10013', 'RA000602', '2154254', 'US-DE', 'GENERAL', 'XTIQ', None, 'ACTIVE', '1988-03-08T00:00:00+00:00', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '2012-06-06T15:53:00+00:00', '2024-04-21T15:18:19+00:00', 'ISSUED', '2025-05-27T14:10:35+00:00', '529900T8BM49AURSDO55', 'FULLY_CORROBORATED', 'RA000602', '2154254', 'RA000628', 1688862, 'CONFORMING')


In [110]:
from llama_index.core.retrievers import SQLRetriever
from typing import List
from llama_index.core.query_pipeline import FnComponent

sql_retriever = SQLRetriever(sql_database)


def get_table_context_and_rows_str(
    query_str: str, table_schema_objs: List[SQLTableSchema]
):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        # first append table info + additional context
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        # also lookup vector index to return relevant table rows
        vector_retriever = vector_index_dict[
            table_schema_obj.table_name
        ].as_retriever(similarity_top_k=2)
        relevant_nodes = vector_retriever.retrieve(query_str)
        if len(relevant_nodes) > 0:
            table_row_context = "\nHere are some relevant example rows (values in the same order as columns above)\n"
            for node in relevant_nodes:
                table_row_context += str(node.get_content()) + "\n"
            table_info += table_row_context

        context_strs.append(table_info)
    return "\n\n".join(context_strs)


table_parser_component = FnComponent(fn=get_table_context_and_rows_str)

In [111]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
)

qp = QP(
    modules={
        "input": InputComponent(),
        "table_retriever": obj_retriever,
        "table_output_parser": table_parser_component,
        "text2sql_prompt": text2sql_prompt,
        "text2sql_llm": llm,
        "sql_output_parser": sql_parser_component,
        "sql_retriever": sql_retriever,
        "response_synthesis_prompt": response_synthesis_prompt,
        "response_synthesis_llm": llm,
    },
    verbose=True,
)

In [112]:
qp.add_link("input", "table_retriever")
qp.add_link("input", "table_output_parser", dest_key="query_str")
qp.add_link(
    "table_retriever", "table_output_parser", dest_key="table_schema_objs"
)
qp.add_link("input", "text2sql_prompt", dest_key="query_str")
qp.add_link("table_output_parser", "text2sql_prompt", dest_key="schema")
qp.add_chain(
    ["text2sql_prompt", "text2sql_llm", "sql_output_parser", "sql_retriever"]
)
qp.add_link(
    "sql_output_parser", "response_synthesis_prompt", dest_key="sql_query"
)
qp.add_link(
    "sql_retriever", "response_synthesis_prompt", dest_key="context_str"
)
qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
qp.add_link("response_synthesis_prompt", "response_synthesis_llm")

In [36]:
response = qp.run(
    query="What is the LEI for CITYGROUP?"
)
print(str(response))

> Running module input with input: 
query: What is the LEI for CITYGROUP?

> Running module table_retriever with input: 
input: What is the LEI for CITYGROUP?

> Running module table_output_parser with input: 
query_str: What is the LEI for CITYGROUP?
table_schema_objs: [SQLTableSchema(table_name='Entity_Legal_Info', context_str='Summary of legal information for various entities including legal names, addresses, registration details, entity status, creation dates, an...

> Running module text2sql_prompt with input: 
query_str: What is the LEI for CITYGROUP?
schema: Table 'Entity_Legal_Info' has columns: Unnamed_0 (INTEGER), LEI (VARCHAR), Entity_LegalName (VARCHAR), Entity_LegalName_xmllang (VARCHAR), Entity_OtherEntityNames_OtherEntityName_1 (VARCHAR), Entity_O...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer. You can order the results 

In [127]:
response = qp.run(
    query="How many relation are direct child of CITIGROUP INC?"
)
print(str(response))

> Running module input with input: 
query: How many relation are direct child of CITIGROUP INC?

> Running module table_retriever with input: 
input: How many relation are direct child of CITIGROUP INC?

> Running module table_output_parser with input: 
query_str: How many relation are direct child of CITIGROUP INC?
table_schema_objs: [SQLTableSchema(table_name='Relationship_Information', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_Relationship...

> Running module text2sql_prompt with input: 
query_str: How many relation are direct child of CITIGROUP INC?
schema: Table 'Relationship_Information' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Relation...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run,

In [122]:
response = qp.run(
    query="How many relation are ultimate child of CITIGROUP INC?"
)
print(str(response))

> Running module input with input: 
query: How many relation are ultimate child of CITIGROUP INC?

> Running module table_retriever with input: 
input: How many relation are ultimate child of CITIGROUP INC?

> Running module table_output_parser with input: 
query_str: How many relation are ultimate child of CITIGROUP INC?
table_schema_objs: [SQLTableSchema(table_name='Relationship_Information', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_Relationship...

> Running module text2sql_prompt with input: 
query_str: How many relation are ultimate child of CITIGROUP INC?
schema: Table 'Relationship_Information' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Relation...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query

In [114]:
response = qp.run(
    query="How many relation are IS_ULTIMATELY_CONSOLIDATED_BY with CITIGROUP INC?"
)
print(str(response))

> Running module input with input: 
query: How many relation are IS_ULTIMATELY_CONSOLIDATED_BY with CITIGROUP INC?

> Running module table_retriever with input: 
input: How many relation are IS_ULTIMATELY_CONSOLIDATED_BY with CITIGROUP INC?

> Running module table_output_parser with input: 
query_str: How many relation are IS_ULTIMATELY_CONSOLIDATED_BY with CITIGROUP INC?
table_schema_objs: [SQLTableSchema(table_name='Relationship_Information', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_Relationship...

> Running module text2sql_prompt with input: 
query_str: How many relation are IS_ULTIMATELY_CONSOLIDATED_BY with CITIGROUP INC?
schema: Table 'Relationship_Information' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Relation...

> Running module text2sql_llm with input: 
messages: Given 

In [126]:
response = qp.run(
    query="how many relations are managing fund for CITIGROUP INC?"
)
print(str(response))

> Running module input with input: 
query: how many relations are managing fund for CITIGROUP INC?

> Running module table_retriever with input: 
input: how many relations are managing fund for CITIGROUP INC?

> Running module table_output_parser with input: 
query_str: how many relations are managing fund for CITIGROUP INC?
table_schema_objs: [SQLTableSchema(table_name='Relationship_Information', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_Relationship...

> Running module text2sql_prompt with input: 
query_str: how many relations are managing fund for CITIGROUP INC?
schema: Table 'Relationship_Information' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Relation...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite q

In [90]:
response = qp.run(
    query="LEI column in Entity_Legal_Info table is associated with which foreign key in relationship_information table"
)
print(str(response))

> Running module input with input: 
query: LEI column in Entity_Legal_Info table is associated with which foreign key in relationship_information table

> Running module table_retriever with input: 
input: LEI column in Entity_Legal_Info table is associated with which foreign key in relationship_information table

> Running module table_output_parser with input: 
query_str: LEI column in Entity_Legal_Info table is associated with which foreign key in relationship_information table
table_schema_objs: [SQLTableSchema(table_name='Relationship_Information', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_Relationship...

> Running module text2sql_prompt with input: 
query_str: LEI column in Entity_Legal_Info table is associated with which foreign key in relationship_information table
schema: Table 'Relationship_Information' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relation

In [95]:
response = qp.run(
    query="how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc"
)
print(str(response))

> Running module input with input: 
query: how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc

> Running module table_retriever with input: 
input: how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc

> Running module table_output_parser with input: 
query_str: how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc
table_schema_objs: [SQLTableSchema(table_name='Relationship_Information', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_Relationship...

> Running module text2sql_prompt with input: 
query_str: how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc
schema: Table 'Relationship_Information' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Relation...

> Running module text2sql_llm with input: 
messages: Given an input questio

In [73]:
response = qp.run(
    query="how many RelationshipType in Relationship Information Table are direct children of CityGroup Inc"
)
print(str(response))

> Running module input with input: 
query: how many RelationshipType in Relationship Information Table are direct children of CityGroup Inc

> Running module table_retriever with input: 
input: how many RelationshipType in Relationship Information Table are direct children of CityGroup Inc

> Running module table_output_parser with input: 
query_str: how many RelationshipType in Relationship Information Table are direct children of CityGroup Inc
table_schema_objs: [SQLTableSchema(table_name='Relationship_Information', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_Relationship...

> Running module text2sql_prompt with input: 
query_str: how many RelationshipType in Relationship Information Table are direct children of CityGroup Inc
schema: Table 'Relationship_Information' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationshi

In [96]:
response = qp.run(
    query="how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc"
)
print(str(response))

> Running module input with input: 
query: how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc

> Running module table_retriever with input: 
input: how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc

> Running module table_output_parser with input: 
query_str: how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc
table_schema_objs: [SQLTableSchema(table_name='Relationship_Information', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_Relationship...

> Running module text2sql_prompt with input: 
query_str: how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc
schema: Table 'Relationship_Information' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Relation...

> Running module text2sql_llm with input: 
messages: Given an input questio

In [97]:
response = qp.run(
    query="how many records are direct child of CityGroup Inc"
)
print(str(response))

> Running module input with input: 
query: how many records are direct child of CityGroup Inc

> Running module table_retriever with input: 
input: how many records are direct child of CityGroup Inc

> Running module table_output_parser with input: 
query_str: how many records are direct child of CityGroup Inc
table_schema_objs: [SQLTableSchema(table_name='Relationship_Information', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_Relationship...

> Running module text2sql_prompt with input: 
query_str: how many records are direct child of CityGroup Inc
schema: Table 'Relationship_Information' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Relation...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then lo